In [121]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

## memory 공유 확인 위한 dummy chain 생성

In [122]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

dummy_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. If order_id is provided, you need to output '조회'.
            If order_id is not provided, you need to classify the customer input message into one of the following two types:
            - 상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'
            - 주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}"),
    ]
)
dummy_prompt.pretty_print()

================================ System Message ================================


            You are a robot that classifies customer input messages into specific types. If order_id is provided, you need to output '조회'.
            If order_id is not provided, you need to classify the customer input message into one of the following two types:
            - 상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'
            - 주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'
            

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

input:{input}
order_id:{order_id}


In [123]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [124]:
dummy_chain = dummy_prompt | model 

In [125]:
from langchain_core.runnables.history import RunnableWithMessageHistory

dumy_chain_with_memory = RunnableWithMessageHistory(
    dummy_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [126]:
response = dumy_chain_with_memory.invoke(
    {"input": "주문 변경하고 싶어", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240517-3"}}
    )
response

Parent run f1f4d9ea-8b7b-486f-90a1-99b09ce02e74 not found for run bb724bcd-d986-4e95-a1b0-0f4820e062ca. Treating as a root run.


AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 148, 'total_tokens': 151}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-34a1df9c-e156-4d60-94b6-eb7ef945c745-0')

In [127]:
store["test_240517-3"].messages

[HumanMessage(content='주문 변경하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 148, 'total_tokens': 151}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-34a1df9c-e156-4d60-94b6-eb7ef945c745-0')]

## 주문 변경/취소 분류 체인

In [128]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

order_change_cancel_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. 
            You need to review the messages in the Messages Placeholder from the latest to the oldest and output either '주문 변경' or '주문 취소'.
            Pay particular attention to the latest HumanMessage when making the classification.
            Ensure to focus on specific keywords for classification:
            - If the message contains words like '취소', '취소하고 싶어', '취소해주세요', classify it as '주문 취소'.
            - If the message contains words like '변경', '바꾸고 싶어', '변경해주세요', classify it as '주문 변경'.
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}"),
    ]
)
order_change_cancel_prompt

ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            You need to review the messages in the Messages Placeholder from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n            Pay particular attention to the latest HumanMessage when making the classification.\n            Ensure to focus on specific keywords for classification:\n            - If the message contains words like '취소', '취소하고 싶어', '취소해주세요', classify it as '주문 취소'.\n            - I

In [129]:
classify_change_or_cancel_chain = order_change_cancel_prompt | model
classify_change_or_cancel_chain

ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            You need to review the messages in the Messages Placeholder from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n            Pay particular attention to the latest HumanMessage when making the classification.\n            Ensure to focus on specific keywords for classification:\n            - If the message contains words like '취소', '취소하고 싶어', '취소해주세요', classify it as '주문 취소'.\n            - I

In [130]:
from langchain_core.runnables.history import RunnableWithMessageHistory

classify_change_or_cancel_chain_with_memory = RunnableWithMessageHistory(
    classify_change_or_cancel_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [131]:
from langchain_core.runnables import RunnableLambda
classify_change_or_cancel_chain_with_memory_lambda = RunnableLambda(
    lambda x: classify_change_or_cancel_chain_with_memory.invoke(x,
                                                                 config={"configurable": {"session_id": "test_240517-3"}})
)
classify_change_or_cancel_chain_with_memory_lambda

RunnableLambda(lambda x: classify_change_or_cancel_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-3'}}))

## 승인 메시지 식별 체인

In [132]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

classify_confirmation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. 
            You need to determine whether the user has responded to the AI's request for approval regarding the action specified in action_type.
            
            To make this determination, the following conditions must be met:
            1. There must be an AIMessage that asks for approval for the action specified in action_type, based on the specified order details.
            2. The user must have explicitly expressed consent in response to the AIMessage asking for approval of the action specified in action_type.
            
            The chat history is provided as a list, where the most recent message is at the end of the list. You should start checking from the most recent message and move backwards.

            Your response must be either 'yes' or 'no'.
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}\naction_type: {action_type}"),
    ]
)
classify_confirmation_prompt

ChatPromptTemplate(input_variables=['action_type', 'chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            You need to determine whether the user has responded to the AI's request for approval regarding the action specified in action_type.\n            \n            To make this determination, the following conditions must be met:\n            1. There must be an AIMessage that asks for approval for the action specified in action_type, based on the specified order details.\n            2. The user m

In [133]:
classify_confirmation_chain = classify_confirmation_prompt | model 
classify_confirmation_chain

ChatPromptTemplate(input_variables=['action_type', 'chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            You need to determine whether the user has responded to the AI's request for approval regarding the action specified in action_type.\n            \n            To make this determination, the following conditions must be met:\n            1. There must be an AIMessage that asks for approval for the action specified in action_type, based on the specified order details.\n            2. The user m

In [134]:
from langchain_core.runnables.history import RunnableWithMessageHistory

classify_confirmation_chain_with_memory = RunnableWithMessageHistory(
    classify_confirmation_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [135]:
from langchain_core.runnables import RunnableLambda

classify_confirmation_chain_with_memory_lambda = RunnableLambda(
    lambda x: classify_confirmation_chain_with_memory.invoke(x,
                                                             config={"configurable": {"session_id": "test_240517-3"}})
)
classify_confirmation_chain_with_memory_lambda

RunnableLambda(lambda x: classify_confirmation_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-3'}}))

## action_type 추가 위한 보조 함수
* 이 부분 다른 방식 가능 여부 찾아봐야

In [136]:
def helper(dict):
    inputs = dict["inputs"]
    action_type =  dict["action_type"]
    action_type = action_type.content
    # action_type = action_type.content
    inputs["action_type"] = action_type
    return inputs

## 종합-입력에 승인 메시지 포함 여부까지 추가

In [137]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

confirmation_chain = (
    {
        "inputs": RunnablePassthrough(),
        "action_type": classify_change_or_cancel_chain_with_memory_lambda
    } 
    | RunnableLambda(helper) 
    | RunnablePassthrough.assign(execution_confirmation=classify_confirmation_chain_with_memory_lambda)
    )
confirmation_chain

{
  inputs: RunnablePassthrough(),
  action_type: RunnableLambda(lambda x: classify_change_or_cancel_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-3'}}))
}
| RunnableLambda(helper)
| RunnableAssign(mapper={
    execution_confirmation: RunnableLambda(lambda x: classify_confirmation_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-3'}}))
  })

이렇게 수정하면 될 듯?
* 흠 아니네?..

In [166]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

confirmation_chain = (
    {
        "inputs": RunnablePassthrough(),
    } 
    | RunnablePassthrough.assign(action_type=classify_change_or_cancel_chain_with_memory_lambda | StrOutputParser()) 
    | RunnablePassthrough.assign(execution_confirmation=classify_confirmation_chain_with_memory_lambda)
    )
confirmation_chain

{
  inputs: RunnablePassthrough()
}
| RunnableAssign(mapper={
    action_type: RunnableLambda(lambda x: classify_change_or_cancel_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-3'}}))
                 | StrOutputParser()
  })
| RunnableAssign(mapper={
    execution_confirmation: RunnableLambda(lambda x: classify_confirmation_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-3'}}))
  })

In [167]:
confirmation_chain.invoke(
    {"input": "주문 변경하고 싶어어",
     "order_id": 7}
)

Parent run be76dad9-2dc4-4987-b227-5bce36fc584b not found for run c0874c53-8afa-4863-9d19-536bf5e410e2. Treating as a root run.


KeyError: "Input to ChatPromptTemplate is missing variables {'order_id', 'input'}.  Expected: ['chat_history', 'input', 'order_id'] Received: ['inputs', 'chat_history']"

## 승인 메시지 작성 체인

In [138]:
from langchain_core.prompts import ChatPromptTemplate

generate_confirm_message_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that generates a confirmation request message based on the action_type and queried_result.
            Create a message to show the queried_result and ask for final confirmation considering action_type.
            The response should be generated in Korean.
            Make sure to clarify that the queried_result is the order detail the user wants to change or cancel, and this is for final confirmation.
            Based on the action_type, ask the user if they want to proceed with the specified action.
            """
        ),
        ("human", "action_type:{action_type}\nqueried_result:{queried_result}")
    ]
)
generate_confirm_message_prompt

ChatPromptTemplate(input_variables=['action_type', 'queried_result'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\n            You are a robot that generates a confirmation request message based on the action_type and queried_result.\n            Create a message to show the queried_result and ask for final confirmation considering action_type.\n            The response should be generated in Korean.\n            Make sure to clarify that the queried_result is the order detail the user wants to change or cancel, and this is for final confirmation.\n            Based on the action_type, ask the user if they want to proceed with the specified action.\n            ')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['action_type', 'queried_result'], template='action_type:{action_type}\nqueried_result:{queried_result}'))])

In [139]:
generate_confirm_message_chain = generate_confirm_message_prompt | model
generate_confirm_message_chain

ChatPromptTemplate(input_variables=['action_type', 'queried_result'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\n            You are a robot that generates a confirmation request message based on the action_type and queried_result.\n            Create a message to show the queried_result and ask for final confirmation considering action_type.\n            The response should be generated in Korean.\n            Make sure to clarify that the queried_result is the order detail the user wants to change or cancel, and this is for final confirmation.\n            Based on the action_type, ask the user if they want to proceed with the specified action.\n            ')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['action_type', 'queried_result'], template='action_type:{action_type}\nqueried_result:{queried_result}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000016EFEDBBD60>, async_

아래 필요 없음! - 모델이 생성한 승인 요청 메시지 저장 안 해도 되니 메모리 사용하지 않아도 됨

In [140]:
from langchain_core.runnables.history import RunnableWithMessageHistory

generate_confirm_message_chain_with_memory = RunnableWithMessageHistory(
    generate_confirm_message_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [141]:
from langchain_core.runnables import RunnableLambda
generate_confirm_message_chain_with_memory_lambda = RunnableLambda(
    lambda x: generate_confirm_message_chain_with_memory.invoke(x,
                                                             config={"configurable": {"session_id": "test_240517-3"}})
)
generate_confirm_message_chain_with_memory_lambda

RunnableLambda(lambda x: generate_confirm_message_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-3'}}))

## 주문 변경 체인

실제 구현 시에는 파일에서 불러와야 할 데이터

In [142]:
products = [
    {
        "product_name": "떡케익5호",
        "quantity": 1,
        "price": 54000
    },
    {
        "product_name": "무지개 백설기 케익",
        "quantity": 1,
        "price": 51500
    },
    {
        "product_name": "미니 백설기",
        "quantity": 35,
        "price": 31500
    },
    {
        "product_name": "개별 모듬팩",
        "quantity": 1,
        "price": 13500
    }
]
products

[{'product_name': '떡케익5호', 'quantity': 1, 'price': 54000},
 {'product_name': '무지개 백설기 케익', 'quantity': 1, 'price': 51500},
 {'product_name': '미니 백설기', 'quantity': 35, 'price': 31500},
 {'product_name': '개별 모듬팩', 'quantity': 1, 'price': 13500}]

주문 변경 체인 동작 점검 위한 가상 데이터

In [143]:
queried_result = [
    {
        "id": 10,
        "created_at": "2024-05-17T12:00:00Z",
        "order_status": "주문 완료",
        "items": [
            {
                "product_name": "떡케익5호",
                "quantity": 1,
                "price": 54000.0
            },
            {
                "product_name": "무지개 백설기 케익",
                "quantity": 1,
                "price": 51500.0
            }
        ]
    },
]
queried_result

[{'id': 10,
  'created_at': '2024-05-17T12:00:00Z',
  'order_status': '주문 완료',
  'items': [{'product_name': '떡케익5호', 'quantity': 1, 'price': 54000.0},
   {'product_name': '무지개 백설기 케익', 'quantity': 1, 'price': 51500.0}]}]

In [144]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

class OrderItem(BaseModel):
    product_name: str = Field(description="The name of the product")
    quantity: int = Field(description="The quantity of the product")
    price: float = Field(description="The price of the product")

class OrderDetails(BaseModel):
    id: int = Field(description="The order ID")
    created_at: str = Field(description="The order creation date")
    order_status: str = Field(description="The current status of the order")
    items: list[OrderItem] = Field(description="The items in the order")

# Set up a parser
parser = PydanticOutputParser(pydantic_object=OrderDetails)

In [145]:
from langchain_core.prompts import PromptTemplate

order_change_prompt = PromptTemplate(
    template="""
    You are a customer service assistant responsible for processing order changes based on customer input and existing order details.
    
    Given the 'queried_result' which contains existing order details and the 'input' which contains the customer's desired order changes, combine these to finalize the order changes.
    
    Follow these steps:
    1. Review the 'queried_result' to understand the original order details.
    2. Review the 'input' to understand the customer's desired changes.
    3. Combine both sets of information to produce the updated order details.
    
    Ensure that the final output includes only the updated items in the order, formatted the same way as the original order, and set the order_status to '주문 변경'.

    {format_instructions}
    products: {products}
    queried_result: {queried_result}
    input: {input}
    """,
    input_variables=["products", "queried_result", "input"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
order_change_prompt

PromptTemplate(input_variables=['input', 'products', 'queried_result'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"id": {"title": "Id", "description": "The order ID", "type": "integer"}, "created_at": {"title": "Created At", "description": "The order creation date", "type": "string"}, "order_status": {"title": "Order Status", "description": "The current status of the order", "type": "string"}, "items": {"title": "Items", "description": "The items in the order", "type": "array", "items": {"$ref": "#/definitions/OrderItem

In [146]:
order_change_chain = order_change_prompt | model | parser
order_change_chain

PromptTemplate(input_variables=['input', 'products', 'queried_result'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"id": {"title": "Id", "description": "The order ID", "type": "integer"}, "created_at": {"title": "Created At", "description": "The order creation date", "type": "string"}, "order_status": {"title": "Order Status", "description": "The current status of the order", "type": "string"}, "items": {"title": "Items", "description": "The items in the order", "type": "array", "items": {"$ref": "#/definitions/OrderItem

In [147]:
def fetch_products(dict):
    print("products", products)
    return products

In [148]:
handle_order_change_chain = RunnablePassthrough.assign(products=RunnableLambda(fetch_products)) | order_change_chain 
handle_order_change_chain

RunnableAssign(mapper={
  products: RunnableLambda(fetch_products)
})
| PromptTemplate(input_variables=['input', 'products', 'queried_result'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"id": {"title": "Id", "description": "The order ID", "type": "integer"}, "created_at": {"title": "Created At", "description": "The order creation date", "type": "string"}, "order_status": {"title": "Order Status", "description": "The current status of the order", "type": "string"}, "items": {"title": "Items", "description": "The items in

# 주문 취소 담당 함수

In [149]:
def cancel_order(dict):
    order_id = dict["order_id"]
    return f"주문{order_id} 취소"

## 주문 변경/취소 라우트

In [150]:
from langchain_core.runnables import RunnableLambda

def change_cancel_route(info):
    print("="*70)
    print("change_cancel_route 함수로 전달된 데이터 -> ", info)
    if "주문 변경" in info["action_type"]:
        print("주문 변경 처리 방향 진입")
        return handle_order_change_chain
    elif "주문 취소" in info["action_type"]:
        return RunnableLambda(cancel_order)


## 승인 메시지 포함 여부에 따른 라우트

In [151]:
# def execution_or_message_route(info):
#     print("="*70)
#     print("execution_or_message_route 함수로 전달된 데이터 -> ", info)
#     if "yes" in info["execution_confirmation"].content.lower():
#         return RunnableLambda(change_cancel_route)
#     else:
#         return generate_confirm_message_chain

현재 로직에서는 confirmation 판단 위해서 승인 요청 메시지가 메모리에 저장돼야 함

In [152]:
def execution_or_message_route(info):
    print("="*70)
    print("execution_or_message_route 함수로 전달된 데이터 -> ", info)
    if "yes" in info["execution_confirmation"].content.lower():
        return RunnableLambda(change_cancel_route)
    else:
        return generate_confirm_message_chain_with_memory_lambda

In [153]:
handle_change_cancel_chain = confirmation_chain | execution_or_message_route
handle_change_cancel_chain

{
  inputs: RunnablePassthrough(),
  action_type: RunnableLambda(lambda x: classify_change_or_cancel_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-3'}}))
}
| RunnableLambda(helper)
| RunnableAssign(mapper={
    execution_confirmation: RunnableLambda(lambda x: classify_confirmation_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-3'}}))
  })
| RunnableLambda(execution_or_message_route)

dummy_chain으로 고객이 주문 취소 요청을 가정한 경우

In [154]:
# store["test_240517-3"].messages

In [155]:
# response = handle_change_cancel_chain.invoke(
#     {"input": '없음',
#      "order_id": 7,
#      "queried_result": queried_result}
# )
# response

In [156]:
# store["test_240517-3"].messages

In [157]:
# response = handle_change_cancel_chain.invoke(
#     {"input": '응',
#      "order_id": 7,
#      "queried_result": queried_result}
# )
# response

dummy_chain으로 고객이 주문 변경 요청을 가정한 경우

In [158]:
store["test_240517-3"].messages

[HumanMessage(content='주문 변경하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 148, 'total_tokens': 151}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-34a1df9c-e156-4d60-94b6-eb7ef945c745-0')]

In [159]:
response = handle_change_cancel_chain.invoke(
    {"input": '없음',
     "order_id": 7,
     "queried_result": queried_result}
)
response

Parent run 3f2c3c69-eed8-453f-8b9d-8df6e9221673 not found for run 302acb87-ebde-4ca5-be1a-cd6b4c14d900. Treating as a root run.
Parent run c11d323c-b25f-458b-b430-8eddf182896b not found for run 6afcb89e-50c4-46c2-a55e-362d69f1fe67. Treating as a root run.
Parent run 0e42378a-25e0-461d-9e82-9f5cd89af319 not found for run d1c02264-01a7-4526-b728-91bb786f00a2. Treating as a root run.


execution_or_message_route 함수로 전달된 데이터 ->  {'input': '없음', 'order_id': 7, 'queried_result': [{'id': 10, 'created_at': '2024-05-17T12:00:00Z', 'order_status': '주문 완료', 'items': [{'product_name': '떡케익5호', 'quantity': 1, 'price': 54000.0}, {'product_name': '무지개 백설기 케익', 'quantity': 1, 'price': 51500.0}]}], 'action_type': '주문 변경', 'execution_confirmation': AIMessage(content='no', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 225, 'total_tokens': 226}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-808a2a56-7d4a-44d8-bf03-cefb8bb997a5-0')}


AIMessage(content='주문 변경을 요청하셨습니다. 변경하고자 하는 주문 내역은 다음과 같습니다:\n주문 ID: 10\n주문일시: 2024-05-17 12:00:00\n주문 상태: 주문 완료\n주문 상품:\n1. 떡케익5호 - 수량: 1 - 가격: 54,000원\n2. 무지개 백설기 케익 - 수량: 1 - 가격: 51,500원\n\n변경하시려는 주문 내역이 맞습니까? 이 내용으로 최종 변경하시겠습니까?', response_metadata={'token_usage': {'completion_tokens': 177, 'prompt_tokens': 226, 'total_tokens': 403}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b47879a9-4fbc-44f5-8592-bca3da8b3211-0')

In [163]:
print(response.content)

주문 변경을 요청하셨습니다. 변경하고자 하는 주문 내역은 다음과 같습니다:
주문 ID: 10
주문일시: 2024-05-17 12:00:00
주문 상태: 주문 완료
주문 상품:
1. 떡케익5호 - 수량: 1 - 가격: 54,000원
2. 무지개 백설기 케익 - 수량: 1 - 가격: 51,500원

변경하시려는 주문 내역이 맞습니까? 이 내용으로 최종 변경하시겠습니까?


In [160]:
queried_result

[{'id': 10,
  'created_at': '2024-05-17T12:00:00Z',
  'order_status': '주문 완료',
  'items': [{'product_name': '떡케익5호', 'quantity': 1, 'price': 54000.0},
   {'product_name': '무지개 백설기 케익', 'quantity': 1, 'price': 51500.0}]}]

In [161]:
# response = handle_change_cancel_chain.invoke(
#     {"input": "떡케익만 주문할게",
#      "order_id": 7,
#      "queried_result": queried_result}
# )
# response

무지개 백설기와 미니 백설기처럼 여러 제품에 포함된 키워드로 주문하니 내 의도와 다른 제품 주문하는 상황 발생

In [164]:
response = handle_change_cancel_chain.invoke(
    {"input": "백설기 케익 하나 더 주문할게",
     "order_id": 7,
     "queried_result": queried_result}
)
response

Parent run 645e79b3-933b-4795-9616-758c5797ad71 not found for run 67b06597-d68c-465f-b6a0-9fc322c6a205. Treating as a root run.
Parent run 58102a13-8851-49e5-86b5-37df33b2551b not found for run aa2404ce-4c4f-4043-af0c-642d967054a4. Treating as a root run.


execution_or_message_route 함수로 전달된 데이터 ->  {'input': '백설기 케익 하나 더 주문할게', 'order_id': 7, 'queried_result': [{'id': 10, 'created_at': '2024-05-17T12:00:00Z', 'order_status': '주문 완료', 'items': [{'product_name': '떡케익5호', 'quantity': 1, 'price': 54000.0}, {'product_name': '무지개 백설기 케익', 'quantity': 1, 'price': 51500.0}]}], 'action_type': '주문 변경', 'execution_confirmation': AIMessage(content='yes', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 471, 'total_tokens': 472}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8599e88d-39c9-4694-956e-7d49d0e88c4c-0')}
change_cancel_route 함수로 전달된 데이터 ->  {'input': '백설기 케익 하나 더 주문할게', 'order_id': 7, 'queried_result': [{'id': 10, 'created_at': '2024-05-17T12:00:00Z', 'order_status': '주문 완료', 'items': [{'product_name': '떡케익5호', 'quantity': 1, 'price': 54000.0}, {'product_name': '무지개 백설기 케익', 'quantity': 1, 'price': 51500.0}]}], 'action_type': '주문 변경', 'execution_con

OrderDetails(id=10, created_at='2024-05-17T12:00:00Z', order_status='주문 변경', items=[OrderItem(product_name='떡케익5호', quantity=1, price=54000.0), OrderItem(product_name='무지개 백설기 케익', quantity=1, price=51500.0), OrderItem(product_name='미니 백설기', quantity=35, price=31500.0), OrderItem(product_name='개별 모듬팩', quantity=1, price=13500.0), OrderItem(product_name='백설기 케익', quantity=1, price=51500.0)])

In [162]:
# store["test_240517-3"].messages